# Data details

In [1]:
from teilab.datasets import TeiLabDataSets

In [2]:
datasets = TeiLabDataSets(verbose=False)
_ = datasets.get_data(password="<PASSWORD1>")
_ = datasets.get_data(password="<PASSWORD2>")

In [3]:
print(f"Group Names   : {datasets.samples._group_names}")
print(f"Group Numbers : {datasets.samples._group_numbers}")

Group Names   : ['SG19378659_257236339458_S001_GE1_1200', 'US91503671_253949442637_S01_GE1_105']
Group Numbers : [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


## Group0

Since there is little data in ``df_meta``, we will omit it here and check with [**Group1**](#Group1)

## Group1

Use data in group1 (`US91503671_253949442637_S01_GE1_105_MMMDD_X_X.txt`)

In [4]:
no = -1
print(
    f"Sample No: {no}",
    f"Filename : {datasets.samples.FileName[no]}",
    f"Condition: {datasets.samples.Condition[no]}",
    sep="\n"
)

Sample No: -1
Filename : US91503671_253949442637_S01_GE1_105_Dec08_2_2.txt
Condition: siVIM-270(2'OMe2-8)


In [5]:
df_meta    = datasets.read_meta(no=no)
df_summary = datasets.read_summary(no=no)
df_data    = datasets.read_data(no=no)

###  Background Subtraction Method

Since the value of `gProcessedSignal` is **"Background Subtracted"** data by _Agilent Scanner_, check what kind of processing was done.

> Reference: [Agilent Feature Extraction 12.0 Reference Guide](https://www.agilent.com/cs/library/usermanuals/public/G4460-90052_FE_RefGuide.pdf)

<table>
  <thead>
    <tr>
      <th>Background Subtraction Method</th>
      <th>Background Subtraction Variable</th>
      <th>SpDe OFF<br>GBA OFF</th>
      <th>SpDe ON<br>GBA OFF</th>
      <th>SpDe OFF<br>GBA ON</th>
      <th>SpDe ON<br>GBA ON</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td rowspan="3">No background subtract</td>
      <td><code>BGUsed</code> =</td>
      <td><code>BGMeanSignal</code></td>
      <td><code>SDSV</code></td>
      <td><code>BGAdjust</code></td>
      <td><code>SDSV</code> + <code>BGAdjust</code></td>
    </tr>
    <tr>
      <td><code>BGSDUsed</code> = </td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
    </tr>
    <tr>
      <td><code>BGSubSignal</code> = </td>
      <td><code>MeanSignal</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
    </tr>
    <tr>
      <td rowspan="3">Local Background</td>
      <td><code>BGUsed</code> = </td>
      <td><code>BGMeanSignal</code></td>
      <td><code>BGMeanSignal</code> + <code>SDSV</code></td>
      <td><code>BGMeanSignal</code> + <code>BGAdjust</code></td>
      <td><code>BGMeanSignal</code> + <code>SDSV</code> + <code>BGAdjust</code></td>
    </tr>
    <tr>
      <td><code>BGSDUsed</code> = </td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
      <td><code>BGPixSDev</code></td>
    </tr>
    <tr>
      <td><code>BGSubSignal</code> = </td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
    </tr>
    <tr>
      <td rowspan="3">Global Background method</td>
      <td><code>BGUsed</code> = </td>
      <td><code>GBGIA</code></td>
      <td><code>GBGIA</code> + <code>SDSV</code></td>
      <td><code>GBGIA</code> + <code>BGAdjust</code></td>
      <td><code>GBGIA</code> + <code>SDSV</code> + <code>BGAdjust</code></td>
    </tr>
    <tr>
      <td><code>BGSDUsed</code> = </td>
      <td><code>GBGISD</code></td>
      <td><code>GBGISD</code></td>
      <td><code>GBGISD</code></td>
      <td><code>GBGISD</code></td>
    </tr>
    <tr>
      <td><code>BGSubSignal</code> = </td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
      <td><code>MeanSignal</code> - <code>BGUsed</code></td>
    </tr>
  </tbody>
</table>

In [6]:
print(f'BGSubtractor_BGSubMethod: {df_meta["BGSubtractor_BGSubMethod"].values[0]}')

BGSubtractor_BGSubMethod: 7


> The `BGSubMethod` of `7` corresponds to "No Background Subtraction method" (see [Table 17 on page 129](https://www.agilent.com/cs/library/usermanuals/public/G4460-90052_FE_RefGuide.pdf).). 

- Global Background Adjustment is turned Off. 
- Spatial Detrending is turned On.

In [7]:
assert all(df_data["gBGUsed"] == df_data["gSpatialDetrendSurfaceValue"])
assert all(abs(df_data["gBGSDUsed"] - df_data["gBGPixSDev"])<1e3)
assert all(abs((df_data["gBGSubSignal"] - (df_data["gMeanSignal"] - df_data["gBGUsed"]))/df_data["gBGSubSignal"]) < 1e4)